In [6]:
import os
from datetime import datetime, timedelta
import pandas as pd
from binance.client import Client
from dotenv import load_dotenv

load_dotenv()
api_key = os.getenv("API_KEY")
api_secret = os.getenv("API_SECRET")

client = Client(api_key, api_secret)

three_months_ago = datetime.now() - timedelta(days=90)

klines = client.get_historical_klines(
    symbol='BTCUSDT',
    interval=Client.KLINE_INTERVAL_5MINUTE,
    start_str=three_months_ago.strftime("%Y-%m-%d %H:%M:%S")
)

df = pd.DataFrame(klines, columns=[
    'Open time', 'Open', 'High', 'Low', 'Close', 'Volume', 'Close time',
    'Quote asset volume', 'Number of trades', 'Taker buy base asset volume',
    'Taker buy quote asset volume', 'Ignore'
])

df['Open time'] = pd.to_datetime(df['Open time'], unit='ms')
price_df = df[['Open time', 'Close']].copy()
price_df['Close'] = pd.to_numeric(price_df['Close'])

print(price_df)

                Open time      Close
0     2025-05-23 15:40:00  109491.76
1     2025-05-23 15:45:00  109311.95
2     2025-05-23 15:50:00  109266.63
3     2025-05-23 15:55:00  108940.00
4     2025-05-23 16:00:00  108911.20
...                   ...        ...
25831 2025-08-21 08:15:00  113765.62
25832 2025-08-21 08:20:00  113809.50
25833 2025-08-21 08:25:00  113760.01
25834 2025-08-21 08:30:00  113780.33
25835 2025-08-21 08:35:00  113734.22

[25836 rows x 2 columns]
